# Plotting: particles

In this notebook, we will load a cosmological simulation and visualize the dark matter particles within it.

## A basic look at the particle positions

In [ ]:
import osyris

path = "osyrisdata/cosmology"
data = osyris.Dataset(100, scale="Mpc", path=path).load('part')

We convert the particle masses to solar masses:

In [ ]:
data['part']['mass'].to('msun')
data

Having loaded the particle data,
we can quickly inspect the distribution of the particles by histogramming their `x` and `y` positions.
We use the particle mass as weights for the histogramming,
to give us a feel for how much mass is contained in one of the pixels of the image below.

In [ ]:
osyris.histogram2d(data["part"]["position"].x,
                   data["part"]["position"].y,
                   data['part']['mass'],
                   resolution=512, norm="log")

## Sorting particles

When particles are loaded, they are by default automatically sorted according to their `identity`.

In [ ]:
data["part"]["identity"][:20].values

It is possible to sort the `Datagroup` according to a new key,
if we wish for instance to sort the particles by AMR level.
This is achieved using the `.sortby()` method of the `Datagroup`.

In [ ]:
data["part"]["levelp"][:20].values

In [ ]:
data["part"].sortby("levelp")
data["part"]["levelp"][:20].values

## 3D rendering

Osyris does not support 3D plotting out of the box,
but a 3D rendering of the particle positions in space can be achieved relatively easily in a Jupyter notebook using the `pythreejs` library.

<div class="alert alert-info">

**Note**

`pythreejs` is not a hard-dependency of Osyris. It needs to be installed separatey with `pip install pythreejs`.

</div>

In [ ]:
import pythreejs as p3

# Create a position buffer geometry
# Only plot one in 10 points to reduce the size of the output
geometry = p3.BufferGeometry(
    attributes={
        'position': p3.BufferAttribute(
            array=data["part"]["position"][::10].values.astype('float32') - 75)
    })
# Create a points material
material = p3.PointsMaterial(color='black', size=1)
# Combine the geometry and material into a Points object
points = p3.Points(geometry=geometry, material=material)

# Create the scene and the renderer
view_width = 700
view_height = 500
camera = p3.PerspectiveCamera(position=[200.0, 0, 0], aspect=view_width/view_height)
scene = p3.Scene(children=[points, camera], background="#DDDDDD")
controller = p3.OrbitControls(controlling=camera)
renderer = p3.Renderer(camera=camera, scene=scene, controls=[controller],
                    width=view_width, height=view_height)
renderer